In [ ]:
import os
import scipy.io
import scipy.misc
import numpy as np
from numpy import expand_dims
import pandas as pd
import PIL
import struct
import cv2
from numpy import expand_dims
import ast
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.patches import Rectangle
from skimage.transform import resize 
import matplotlib


add to path

In [ ]:
path = '/kaggle/input/siim-covid19-detection/'
os.listdir(path)

load data 

In [ ]:
train_image = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
train_study = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")
samp_submi = pd.read_csv(path+'sample_submission.csv')

In [ ]:
train_image.head(10)

In [ ]:
train_image.tail(10)

In [ ]:
train_study.head(10)

In [ ]:
train_study.tail(10)

In [ ]:
print('Number train images samples:', len(train_image))
print('Number train study samples:', len(train_study))

In [ ]:
import pydicom as dicom
# Define image path of the example
path_train = path+'train/'+train_image.loc[0, 'StudyInstanceUID']+'/'+'81456c9c5423'+'/'
# Extract image name of the example
img_id = train_image.loc[0, 'id'].replace('_image', '.dcm')
# Load dicom file
data_file = dicom.dcmread(path_train+img_id)
# Extract image data of the dicom file
img = data_file.pixel_array

In [ ]:
print(data_file)

In [ ]:
print('Image shape:', img.shape)

In [ ]:
boxes = ast.literal_eval(train_image.loc[0, 'boxes'])
boxes

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 4))

for box in boxes:
    p = matplotlib.patches.Rectangle((box['x'], box['y']), box['width'], box['height'],
                                     ec='r', fc='none', lw=2.)
    ax.add_patch(p)
ax.imshow(img, cmap='gray')
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(40, 40))
fig.subplots_adjust(hspace = .1, wspace=.1)
axs = axs.ravel()

for row in range(9):
    study = train_image.loc[row, 'StudyInstanceUID']
    path_in = path+'train/'+study+'/'
    folder = os.listdir(path_in)
    path_file = path_in+folder[0]
    filename = os.listdir(path_file)[0]
    file_id = filename.split('.')[0]
    
    data_file = dicom.dcmread(path_file+'/'+file_id+'.dcm')
    img = data_file.pixel_array
    if (train_image.loc[row, 'boxes']!=train_image.loc[row, 'boxes']) == False:
        boxes = ast.literal_eval(train_image.loc[row, 'boxes'])
    
        for box in boxes:
            p = matplotlib.patches.Rectangle((box['x'], box['y']), box['width'], box['height'],
                                     ec='r', fc='none', lw=2.)
            axs[row].add_patch(p)
    axs[row].imshow(img, cmap='gray')
    axs[row].set_title(train_image.loc[row, 'label'].split(' ')[0])
    axs[row].set_xticklabels([])
    axs[row].set_yticklabels([])

In [ ]:
label_dict = {0: 'none', 1: 'simple_opacity', 2: 'double_opacity'}

In [ ]:
def split_label(s):
    split_string = s.split(' ')
    if len(split_string)==6 and 'none' in split_string:
        return 0
    elif len(split_string)==6 and 'opacity' in split_string:
        return 1
    else:
        return 2

In [ ]:
train_image['category'] = train_image['label'].apply(split_label)

In [ ]:
train_image.head(20)

In [ ]:
train_image.tail(20)